# "버거 지수"는 진짜 도시의 발전 수준을 반영할까?
 - 지도 그리기 관련 공부 중 <a href="http://nbviewer.jupyter.org/gist/hyeshik/cf9f3d7686e07eedbfda?revision=6"> 참고 </a>
 - "한 도시의 발전 수준은 (버거킹의 개수+맥도날드의 개수+KFC의 개수)/롯데리아의 개수를 계산하여 높게 나올수록 더 발전된 도시라고 할 수 있다
 - $\xi=\frac {B+M+K} {L}  $

In [1]:
import urllib.request
import json
import pandas as pd
import bs4

## 1. 각 브랜드 매장 데이터 
### 1.1 버거킹 매장

In [3]:
response = urllib.request.urlopen('http://www.burgerking.co.kr/api/store/searchmap/empty/?areacd=')
bgk_data = json.loads(response.read().decode('utf-8'))
bgk_tbl = pd.DataFrame(bgk_data)

In [4]:
bgk_tbl.head()

,AllHour,CloseCleaning,ClosePeakSeason,CloseWeekday,Delivery,DriveThrough,Morning,NewAddr,NewAddr2,OpenTime,PhoneNumber,PointX,PointY,StoreNM,StoreSQ
0,N,,,22:00,Y,N,N,서울특별시 금천구 가산디지털1로,168 우림라이온스밸리 A동,09:00,02-853-0332,37.480896,126.882133,가산디지털점,1
1,N,,/ 토: 10시 ~ 23시 / 일: 10시 ~ 22시,22시 / 금: 11시~23시,N,N,N,서울특별시 금천구 디지털로10길,9 HYHILL 6층,월~목: 11시,02-2136-9962,37.477620,126.889053,가산현대시티아울렛점,2
2,N,,,23:30,N,N,N,서울특별시 강서구 양천로,559 이마트3층 (가양동),10:00,02-3664-0221,37.558193,126.861816,가양이마트점,3
3,N,,,23:00,Y,N,N,서울시 서대문구 북가좌동,487 성공타워 1층,09:00,02-303-0392,37.570650,126.912750,가재울뉴타운점,275
4,Y,Cleaning Day(Close): 매월 셋째주 월요일 02:00~08:00,,,N,N,Y,서울특별시 서초구 잠원로,69,,02-595-9042,37.509400,127.007309,강남NC점,11


In [17]:
bgk_locs = pd.DataFrame(bgk_tbl['NewAddr'].apply(lambda x:x.split()[:2]).tolist(), columns=["d1","d2"])

In [19]:
bgk_locs.head()

,d1,d2
0,서울특별시,금천구
1,서울특별시,금천구
2,서울특별시,강서구
3,서울시,서대문구
4,서울특별시,서초구


In [20]:
bgk_locs['d1'].unique() # 광역시 이름 확인. 섞여서 있다.

array(['서울특별시', '서울시', '경기도', '수원시', '안산시', '인천광역시', '인천시', '인천', '강릉시',
       '강원도', '충청남도', '충남', '대전광역시', '대전시', '충청북도', '청주시', '세종특별자치시',
       '부산광역시', '부산시', '울산광역시', '대구광역시', '대구시', '경북', '경상북도', '포항시',
       '경상남도', '경남', '전라남도', '전남', '광주광역시', '전라북도', '전북', '제주특별자치도'], dtype=object)

In [21]:
d1_aliases = """서울시:서울특별시 충남:충청남도 강원:강원도 경기:경기도 충북:충청북도 경남:경상남도 경북:경상북도
전남:전라남도 전북:전라북도 제주도:제주특별자치도 제주:제주특별자치도 대전시:대전광역시 대구시:대구광역시 인천시:인천광역시
광주시:광주광역시 울산시:울산광역시"""

In [32]:
d1_aliases = dict(aliasset.split(":") for aliasset in d1_aliases.split())

In [37]:
bgk_locs['d1'] = bgk_locs['d1'].apply(lambda v: d1_aliases.get(v, v))

In [39]:
bgk_locs['d1'].unique()

array(['서울특별시', '경기도', '수원시', '안산시', '인천광역시', '인천', '강릉시', '강원도', '충청남도',
       '대전광역시', '충청북도', '청주시', '세종특별자치시', '부산광역시', '부산시', '울산광역시', '대구광역시',
       '경상북도', '포항시', '경상남도', '전라남도', '광주광역시', '전라북도', '제주특별자치도'], dtype=object)

In [44]:
d1_aliases2 = """수원시:경기도 안산시:경기도 인천:인천광역시 강릉시:강원도 청주시:충청남도 부산시:부산광역시 포항시:경상북도"""

In [45]:
d1_aliases2 = dict(aliasset.split(":") for aliasset in d1_aliases2.split())
bgk_locs['d1'] = bgk_locs['d1'].apply(lambda v: d1_aliases2.get(v, v))

In [46]:
bgk_locs['d1'].unique()

array(['서울특별시', '경기도', '인천광역시', '강원도', '충청남도', '대전광역시', '충청북도', '세종특별자치시',
       '부산광역시', '울산광역시', '대구광역시', '경상북도', '경상남도', '전라남도', '광주광역시', '전라북도',
       '제주특별자치도'], dtype=object)

In [47]:
bgk_locs['d2'].unique()

array(['금천구', '강서구', '서대문구', '서초구', '강남구', '광진구', '성북구', '마포구', '관악구',
       '구로구', '용산구', '강동구', '동작구', '영등포구', '중구', '양천구', '송파구', '강북구',
       '은평구', '안산시', '도봉구', '안암로', '종로구', '중랑구', '동대문구', '고양시', '과천시',
       '수원시', '광명시', '광주시', '구리시', '파주시', '김포시', '동두천시', '화성시', '부천시',
       '성남시', '군포시', '영통구', '용인시', '단원구', '여주군', '오산시', '의정부시', '이천시',
       '안양시', '평택시', '하남시', '남구', '부평구', '연수구', '송도국제대로', '계양구', '남동구',
       '서구', '하슬라로', '홍천군', '속초시', '원주시', '춘천시', '당진시', '서산시', '아산시',
       '천안시', '유성구', '동구', '청주시', '흥덕구', '청원군', '충주시', None, '사상구', '연제구',
       '부산진구', '해운대구', '수성구', '북구', '달서구', '달성군', '상주시', '경산시', '경주시',
       '구미시', '김천시', '안동시', '포항시', '거제시', '창원시', '김해시', '진주시', '광양시',
       '나주시', '목포시', '순천시', '여수시', '무안군', '광산구', '군산시', '익산시', '전주시',
       '정읍시', '제주시'], dtype=object)

In [63]:
#B = pd.DataFrame({'cnt':bgk_locs.groupby(['d1','d2']).size()}).reset_index()
B = (bgk_locs['d1'] + ' ' + bgk_locs['d2']).value_counts()
B.head()

서울특별시 강남구    19
경기도 수원시      11
서울특별시 서초구     9
경기도 성남시       7
서울특별시 중구      7
dtype: int64

## 1.2 맥도날드 
 - 여기부터 코드가 현재는 에러난다.. 새로 짜기 귀찮은데... 나중에 이코드는 다시 해야지..

In [64]:
#MCDONALDS_URL = 'http://www.mcdonalds.co.kr/www/kor/findus/district.do?sSearch_yn=Y&skey=2&pageIndex={page}&skeyword={location}'

In [74]:
def search_mcdonalds_stores_one_page(location, page):
    response = urllib.request.urlopen(
        MCDONALDS_URL.format(location=urllib.parse.quote(location.encode('utf-8')), page=page))
    mcd_data = response.read().decode('utf-8')
    soup = bs4.BeautifulSoup(mcd_data)
    
    ret = []
    for storetag in soup.findAll('dl', attrs={'class': 'clearFix'}):
        storename = storetag.findAll('a')[0].contents[-1].strip()
        storeaddr = storetag.findAll('dd', attrs={'class': 'road'})[0].contents[0].split(']')[1]
        storeaddr_district = storeaddr.split()[:2]
        ret.append([storename] + storeaddr_district)

    return pd.DataFrame(ret, columns=('store', 'd1', 'd2')) if ret else None

# 여러 페이지를 쭉 찾아서 안 나올 때 까지 합친다.
def search_mcdonalds_stores(location):
    from itertools import count
    returns = ""
    try:
        found = []
        for pg in count():
            foundinpage = search_mcdonalds_stores_one_page(location, pg+1)
            if foundinpage is None:
                break
            found.append(foundinpage)
        
        returns = pd.concat(found)
    except:
        print(location + "없네")

    return returns

 - 버거킹에서 나온 시도명으로 이용